In [1]:
import pandas as pd
import numpy as np
import sys
import os

In [2]:
train=pd.read_parquet('../Data/train_data.parquet', engine='pyarrow')

In [3]:
val=pd.read_parquet('../Data/val_data.parquet', engine='pyarrow')

In [4]:
train.shape,val.shape

((548319, 371), (221845, 371))

In [5]:
train

,id2,id3,id4,id5,y,f1,f2,f3,f4,f5,...,f357,f358,f359,f360,f361,f362,f363,f364,f365,f366
0,1366776,189706075,2023-11-02 22:22:00.042,2023-11-02,0,1.0,None,None,None,None,...,None,-9999.0,0.0,None,28.0,0.0,0.0,337.0,0.0,0.0
1,1366776,89227,2023-11-01 23:51:24.999,2023-11-01,0,1.0,None,None,None,None,...,None,None,0.0,None,87.0,0.0,0.0,1010.0,2.0,0.0019801980198019
2,1366776,35046,2023-11-01 00:30:59.797,2023-11-01,0,1.0,None,None,None,None,...,None,None,0.0,None,23.0,0.0,0.0,1010.0,2.0,0.0019801980198019
3,1366776,6275451,2023-11-02 22:21:32.261,2023-11-02,0,1.0,None,None,None,None,...,None,-9999.0,0.0,None,277.0,1.0,0.003610108303249,337.0,0.0,0.0
4,1366776,78053,2023-11-02 22:21:34.799,2023-11-02,0,1.0,None,None,None,None,...,None,-9999.0,0.0,None,359.0,0.0,0.0,337.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548314,1896641,87731,2023-11-02 08:14:21.524,2023-11-02,0,None,None,None,None,42.0,...,0.0021005417064525,0.0404038812929242,0.0,None,90.0,1.0,0.0111111111111111,282.0,1.0,0.0035460992907801
548315,1896641,505604,2023-11-02 08:14:24.458,2023-11-02,0,None,None,None,None,42.0,...,-0.0005843979733278,0.0506497784165768,0.0,None,33.0,0.0,0.0,58.0,1.0,0.0172413793103448
548316,1896641,25212,2023-11-02 08:14:25.748,2023-11-02,0,None,None,None,None,42.0,...,0.0003542232402053,0.0498707994939233,0.0,None,33.0,0.0,0.0,58.0,1.0,0.0172413793103448
548317,1900765,95157,2023-11-02 06:08:25.900,2023-11-02,0,None,None,None,None,None,...,None,None,0.0,None,None,None,None,None,None,None


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548319 entries, 0 to 548318
Columns: 371 entries, id2 to f366
dtypes: datetime64[ns](1), object(370)
memory usage: 1.5+ GB


In [7]:
data_dict=pd.read_csv('../Data/data_dictionary.csv')

In [8]:
data_dict

,masked_column,Description,Type
0,id1,Primary Key(masked),Key
1,id2,Customer ID (masked),-
2,id3,Offer ID,Categorical
3,id4,Event timestamp,Numerical
4,id5,Event Date,Numerical
...,...,...,...
367,f362,Total clicks on the incoming offer's industry ...,Numerical
368,f363,CTR on the incoming offer's industry for the g...,Numerical
369,f364,CM’s past 6 moth impressions on relevant offers,Numerical
370,f365,CM’s past 6 moth clicks on relevant offers,Numerical


In [9]:
# now for all the columns in data_dict with type Numerical convert those to float in train, val 
for col in data_dict[data_dict['Type']=='Numerical']['masked_column']:
    if(col=='id4' or col=='id5'):
        continue
    if col not in train.columns or col not in val.columns:
        print(f"Column {col} is missing in one of the datasets.")
        continue
    train[col] = train[col].astype(float)
    val[col] = val[col].astype(float)

Column f376 is missing in one of the datasets.


In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548319 entries, 0 to 548318
Columns: 371 entries, id2 to f366
dtypes: datetime64[ns](1), float64(270), object(100)
memory usage: 1.5+ GB


In [11]:
# train.drop(['id5'], axis=1, inplace=True)
# val.drop(['id5'], axis=1, inplace=True)

In [12]:
data_dict['Type'].value_counts()

Type
Numerical          273
One hot encoded     84
Categorical         12
Key                  1
-                    1
Label                1
Name: count, dtype: int64

In [13]:
for col in data_dict[data_dict['Type']=='One hot encoded']['masked_column']:
    if train[col].isnull().any() or val[col].isnull().any():
        # fill missing values with 1
        train[col].fillna(1, inplace=True)
        val[col].fillna(1, inplace=True)

/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_32636/1908783856.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train[col].fillna(1, inplace=True)
/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_32636/1908783856.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

In [14]:
# change the dtype of one hot encoded columns to int
for col in data_dict[data_dict['Type']=='One hot encoded']['masked_column']:
    if col in train.columns and col in val.columns:
        # First convert to float to handle string values like '0.0'
        train[col] = pd.to_numeric(train[col], errors='coerce')
        val[col] = pd.to_numeric(val[col], errors='coerce')
        
        # Fill any NaN values that might have been created
        train[col] = train[col].fillna(0)
        val[col] = val[col].fillna(0)
        
        # Now convert to int
        train[col] = train[col].astype(np.int8)
        val[col] = val[col].astype(np.int8)

In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548319 entries, 0 to 548318
Columns: 371 entries, id2 to f366
dtypes: datetime64[ns](1), float64(270), int8(84), object(16)
memory usage: 1.2+ GB


In [16]:
train['f42'].value_counts()

f42
R    153405
G    119224
P     28833
S     18592
Name: count, dtype: int64

In [17]:
for col in data_dict[data_dict['Type']=='Categorical']['masked_column']:
    print(col)

id3
f42
f48
f50
f52
f53
f54
f55
f56
f57
f349
f354


In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548319 entries, 0 to 548318
Columns: 371 entries, id2 to f366
dtypes: datetime64[ns](1), float64(270), int8(84), object(16)
memory usage: 1.2+ GB


In [19]:
# show the object columns in train
object_cols = train.select_dtypes(include=['object']).columns
print("Object columns in train:")
print(object_cols)

Object columns in train:
Index(['id2', 'id3', 'id4', 'y', 'f42', 'f48', 'f50', 'f52', 'f53', 'f54',
       'f55', 'f56', 'f57', 'f218', 'f349', 'f354'],
      dtype='object')


In [20]:
# convert these 'f42', 'f48', 'f50', 'f52', 'f53', 'f54','f55', 'f56', 'f57', 'f218', 'f349', 'f354' to categorical
for col in object_cols:
    if col in train.columns and col in val.columns :
        train[col] = train[col].astype('category')
        val[col] = val[col].astype('category')

In [21]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548319 entries, 0 to 548318
Columns: 371 entries, id2 to f366
dtypes: category(16), datetime64[ns](1), float64(270), int8(84)
memory usage: 1.2 GB


In [22]:
# convert y to int8
train['y'] = train['y'].astype('int8')
val['y'] = val['y'].astype('int8')


In [23]:
# change the dtype of f349 to int
train['f349'] = train['f349'].astype(int)
val['f349'] = val['f349'].astype(int)

In [24]:
# Create sine and cosine features for cyclic encoding of f349
train['f349_sin'] = np.sin(2 * np.pi * train['f349'] / 7)  # assuming 7 day cycle
train['f349_cos'] = np.cos(2 * np.pi * train['f349'] / 7)

val['f349_sin'] = np.sin(2 * np.pi * val['f349'] / 7)
val['f349_cos'] = np.cos(2 * np.pi * val['f349'] / 7)

/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_32636/3820020391.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train['f349_sin'] = np.sin(2 * np.pi * train['f349'] / 7)  # assuming 7 day cycle
/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_32636/3820020391.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train['f349_cos'] = np.cos(2 * np.pi * train['f349'] / 7)
/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_32636/3820020391.py:5: PerformanceWarning: DataFrame is highly fragmented.  Thi

In [25]:

# Drop the original f349 column
train.drop('f349', axis=1, inplace=True)
val.drop('f349', axis=1, inplace=True)

In [26]:
# convert id4 to datetime
train['id4'] = pd.to_datetime(train['id4'], errors='coerce')
val['id4'] = pd.to_datetime(val['id4'], errors='coerce')

In [27]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548319 entries, 0 to 548318
Columns: 372 entries, id2 to f349_cos
dtypes: category(13), datetime64[ns](2), float64(272), int8(85)
memory usage: 1.2 GB


In [28]:
train

,id2,id3,id4,id5,y,f1,f2,f3,f4,f5,...,f359,f360,f361,f362,f363,f364,f365,f366,f349_sin,f349_cos
0,1366776,189706075,2023-11-02 22:22:00.042,2023-11-02,0,1.0,NaN,NaN,NaN,NaN,...,0.0,NaN,28.0,0.0,0.000000,337.0,0.0,0.000000,-0.974928,-0.222521
1,1366776,89227,2023-11-01 23:51:24.999,2023-11-01,0,1.0,NaN,NaN,NaN,NaN,...,0.0,NaN,87.0,0.0,0.000000,1010.0,2.0,0.001980,-0.433884,-0.900969
2,1366776,35046,2023-11-01 00:30:59.797,2023-11-01,0,1.0,NaN,NaN,NaN,NaN,...,0.0,NaN,23.0,0.0,0.000000,1010.0,2.0,0.001980,-0.433884,-0.900969
3,1366776,6275451,2023-11-02 22:21:32.261,2023-11-02,0,1.0,NaN,NaN,NaN,NaN,...,0.0,NaN,277.0,1.0,0.003610,337.0,0.0,0.000000,-0.974928,-0.222521
4,1366776,78053,2023-11-02 22:21:34.799,2023-11-02,0,1.0,NaN,NaN,NaN,NaN,...,0.0,NaN,359.0,0.0,0.000000,337.0,0.0,0.000000,-0.974928,-0.222521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548314,1896641,87731,2023-11-02 08:14:21.524,2023-11-02,0,NaN,NaN,NaN,NaN,42.0,...,0.0,NaN,90.0,1.0,0.011111,282.0,1.0,0.003546,-0.974928,-0.222521
548315,1896641,505604,2023-11-02 08:14:24.458,2023-11-02,0,NaN,NaN,NaN,NaN,42.0,...,0.0,NaN,33.0,0.0,0.000000,58.0,1.0,0.017241,-0.974928,-0.222521
548316,1896641,25212,2023-11-02 08:14:25.748,2023-11-02,0,NaN,NaN,NaN,NaN,42.0,...,0.0,NaN,33.0,0.0,0.000000,58.0,1.0,0.017241,-0.974928,-0.222521
548317,1900765,95157,2023-11-02 06:08:25.900,2023-11-02,0,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.974928,-0.222521


# Handling Null Values

In [29]:
# print a table showing null values in each column percentage and sort in decreasing order like a table


null_values = train.isnull().sum() / len(train) * 100
null_values = null_values.sort_values(ascending=False)
print(null_values)

f112        100.000000
f135        100.000000
f122        100.000000
f136        100.000000
f80          99.985775
               ...    
f269          0.000000
f270          0.000000
f271          0.000000
f272          0.000000
f349_cos      0.000000
Length: 372, dtype: float64


In [30]:
# show the columns with more than 70 percent null values in train
train.columns[train.isnull().mean() > 0.7]

Index(['f3', 'f4', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20',
       'f21', 'f29', 'f33', 'f34', 'f35', 'f36', 'f37', 'f40', 'f64', 'f66',
       'f70', 'f79', 'f80', 'f81', 'f84', 'f88', 'f92', 'f112', 'f114', 'f116',
       'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f135', 'f136', 'f154',
       'f176', 'f187', 'f188', 'f189', 'f190', 'f191', 'f192', 'f193', 'f194',
       'f195', 'f196', 'f197', 'f205', 'f218', 'f220', 'f221', 'f360'],
      dtype='object')

In [31]:
# drop these columns in both train and val
cols_to_drop = train.columns[train.isnull().mean() > 0.7]
train.drop(cols_to_drop, axis=1, inplace=True)
val.drop(cols_to_drop, axis=1, inplace=True)

In [32]:
train

,id2,id3,id4,id5,y,f1,f2,f5,f6,f7,...,f358,f359,f361,f362,f363,f364,f365,f366,f349_sin,f349_cos
0,1366776,189706075,2023-11-02 22:22:00.042,2023-11-02,0,1.0,NaN,NaN,NaN,NaN,...,-9999.000000,0.0,28.0,0.0,0.000000,337.0,0.0,0.000000,-0.974928,-0.222521
1,1366776,89227,2023-11-01 23:51:24.999,2023-11-01,0,1.0,NaN,NaN,NaN,NaN,...,NaN,0.0,87.0,0.0,0.000000,1010.0,2.0,0.001980,-0.433884,-0.900969
2,1366776,35046,2023-11-01 00:30:59.797,2023-11-01,0,1.0,NaN,NaN,NaN,NaN,...,NaN,0.0,23.0,0.0,0.000000,1010.0,2.0,0.001980,-0.433884,-0.900969
3,1366776,6275451,2023-11-02 22:21:32.261,2023-11-02,0,1.0,NaN,NaN,NaN,NaN,...,-9999.000000,0.0,277.0,1.0,0.003610,337.0,0.0,0.000000,-0.974928,-0.222521
4,1366776,78053,2023-11-02 22:21:34.799,2023-11-02,0,1.0,NaN,NaN,NaN,NaN,...,-9999.000000,0.0,359.0,0.0,0.000000,337.0,0.0,0.000000,-0.974928,-0.222521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548314,1896641,87731,2023-11-02 08:14:21.524,2023-11-02,0,NaN,NaN,42.0,37.0,11.0,...,0.040404,0.0,90.0,1.0,0.011111,282.0,1.0,0.003546,-0.974928,-0.222521
548315,1896641,505604,2023-11-02 08:14:24.458,2023-11-02,0,NaN,NaN,42.0,37.0,11.0,...,0.050650,0.0,33.0,0.0,0.000000,58.0,1.0,0.017241,-0.974928,-0.222521
548316,1896641,25212,2023-11-02 08:14:25.748,2023-11-02,0,NaN,NaN,42.0,37.0,11.0,...,0.049871,0.0,33.0,0.0,0.000000,58.0,1.0,0.017241,-0.974928,-0.222521
548317,1900765,95157,2023-11-02 06:08:25.900,2023-11-02,0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,-0.974928,-0.222521


In [33]:
null_values = train.isnull().sum() / len(train) * 100
null_values = null_values.sort_values(ascending=False)
print(null_values)

f82         67.171118
f48         64.939387
f206        63.767807
f207        63.767807
f1          63.551874
              ...    
f269         0.000000
f270         0.000000
f271         0.000000
f272         0.000000
f349_cos     0.000000
Length: 316, dtype: float64


In [34]:
# save these dataframe as train_cleaned, val_cleaned
train_cleaned = train.copy()
val_cleaned = val.copy()


train_cleaned.to_parquet('../Data/train_cleaned.parquet', index=False)
val_cleaned.to_parquet('../Data/val_cleaned.parquet', index=False)


In [35]:
train

,id2,id3,id4,id5,y,f1,f2,f5,f6,f7,...,f358,f359,f361,f362,f363,f364,f365,f366,f349_sin,f349_cos
0,1366776,189706075,2023-11-02 22:22:00.042,2023-11-02,0,1.0,NaN,NaN,NaN,NaN,...,-9999.000000,0.0,28.0,0.0,0.000000,337.0,0.0,0.000000,-0.974928,-0.222521
1,1366776,89227,2023-11-01 23:51:24.999,2023-11-01,0,1.0,NaN,NaN,NaN,NaN,...,NaN,0.0,87.0,0.0,0.000000,1010.0,2.0,0.001980,-0.433884,-0.900969
2,1366776,35046,2023-11-01 00:30:59.797,2023-11-01,0,1.0,NaN,NaN,NaN,NaN,...,NaN,0.0,23.0,0.0,0.000000,1010.0,2.0,0.001980,-0.433884,-0.900969
3,1366776,6275451,2023-11-02 22:21:32.261,2023-11-02,0,1.0,NaN,NaN,NaN,NaN,...,-9999.000000,0.0,277.0,1.0,0.003610,337.0,0.0,0.000000,-0.974928,-0.222521
4,1366776,78053,2023-11-02 22:21:34.799,2023-11-02,0,1.0,NaN,NaN,NaN,NaN,...,-9999.000000,0.0,359.0,0.0,0.000000,337.0,0.0,0.000000,-0.974928,-0.222521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548314,1896641,87731,2023-11-02 08:14:21.524,2023-11-02,0,NaN,NaN,42.0,37.0,11.0,...,0.040404,0.0,90.0,1.0,0.011111,282.0,1.0,0.003546,-0.974928,-0.222521
548315,1896641,505604,2023-11-02 08:14:24.458,2023-11-02,0,NaN,NaN,42.0,37.0,11.0,...,0.050650,0.0,33.0,0.0,0.000000,58.0,1.0,0.017241,-0.974928,-0.222521
548316,1896641,25212,2023-11-02 08:14:25.748,2023-11-02,0,NaN,NaN,42.0,37.0,11.0,...,0.049871,0.0,33.0,0.0,0.000000,58.0,1.0,0.017241,-0.974928,-0.222521
548317,1900765,95157,2023-11-02 06:08:25.900,2023-11-02,0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,-0.974928,-0.222521


In [36]:
val

,id2,id3,id4,id5,y,f1,f2,f5,f6,f7,...,f358,f359,f361,f362,f363,f364,f365,f366,f349_sin,f349_cos
0,1372036,189706075,2023-11-03 06:28:33.012,2023-11-03,0,67.0,NaN,NaN,NaN,NaN,...,-9999.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,-0.781831,0.62349
1,1372036,79018,2023-11-03 06:28:32.659,2023-11-03,0,67.0,NaN,NaN,NaN,NaN,...,0.047844,0.0,1.0,0.0,0.0,NaN,NaN,NaN,-0.781831,0.62349
2,1377802,803982,2023-11-03 07:05:12.824,2023-11-03,0,90.0,56.0,75.0,60.0,35.0,...,-9999.000000,0.0,1.0,0.0,0.0,75.0,0.0,0.000000,-0.781831,0.62349
3,1377802,66930,2023-11-03 07:05:26.149,2023-11-03,0,90.0,56.0,75.0,60.0,35.0,...,0.044326,0.0,4.0,0.0,0.0,111.0,0.0,0.000000,-0.781831,0.62349
4,1377802,57615,2023-11-03 07:05:25.608,2023-11-03,0,90.0,56.0,75.0,60.0,35.0,...,0.051192,0.0,4.0,0.0,0.0,127.0,0.0,0.000000,-0.781831,0.62349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221840,1858754,7114,2023-11-03 13:34:49.413,2023-11-03,0,NaN,NaN,43.0,37.0,30.0,...,0.055556,0.0,270.0,0.0,0.0,3636.0,15.0,0.004125,-0.781831,0.62349
221841,1858754,93527,2023-11-03 15:29:25.890,2023-11-03,0,NaN,NaN,43.0,37.0,30.0,...,0.085184,0.0,270.0,0.0,0.0,3636.0,15.0,0.004125,-0.781831,0.62349
221842,1889036,430736,2023-11-03 08:35:58.602,2023-11-03,0,13.0,NaN,NaN,32.0,NaN,...,-9999.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,-0.781831,0.62349
221843,1889036,184034,2023-11-03 08:35:57.298,2023-11-03,0,13.0,NaN,NaN,32.0,NaN,...,0.044215,0.0,1.0,0.0,0.0,NaN,NaN,NaN,-0.781831,0.62349


In [37]:
train.shape

(548319, 316)

In [38]:
val.shape

(221845, 316)